In [3]:
#Import Beautiful Soup and Requests Libraries
from bs4 import BeautifulSoup
import requests

In [4]:
#Request link to wikipedia page in text format
Canada_Postal_Link = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

In [5]:
#Apply Beautiful Soup function
soupcanada = BeautifulSoup(Canada_Postal_Link)

In [10]:
#Search for wikitable sortable which contains the table details
wiki_table = soupcanada.find("table", {"class":"wikitable sortable"})

In [11]:
postcode = []
borough = []
neighborhood = []

In [12]:
#Append data into newly created variables
for row in wiki_table.findAll("tr"):
    cells = row.findAll("td")
    if len(cells) == 3:
        postcode.append(cells[0].find(text = True))
        borough.append(cells[1].find(text = True))
        neighborhood.append(cells[2].find(text = True))

In [13]:
#Pandas for data frame 
import pandas as pd

In [14]:
#Create dataframe columns
table_df = pd.DataFrame(columns = ["Postalcode", "Borough", "Neighbourhood"])

In [15]:
#Insert data into newly created dataframe
table_df["Postalcode"] = postcode

In [16]:
table_df["Borough"] = borough

In [17]:
table_df["Neighbourhood"] = neighborhood

In [18]:
table_df.set_index("Postalcode", inplace = True)

In [19]:
table_df.head()

,Borough,Neighbourhood
Postalcode,,
M1A,Not assigned,Not assigned
M2A,Not assigned,Not assigned
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,Harbourfront


In [20]:
#Remove rows where Borough value is null
table_df = table_df[table_df.Borough != "Not assigned"]

In [21]:
#Strip \n from string value
table_df["Neighbourhood"] = table_df["Neighbourhood"].str.strip("\n")

In [22]:
#Replace Neighbourhood with not assigned value with Borough
table_df["Neighbourhood"] = table_df["Neighbourhood"].replace(["Not assigned"], "Queens Park")

In [23]:
table_df.head(10)

,Borough,Neighbourhood
Postalcode,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,Harbourfront
M5A,Downtown Toronto,Regent Park
M6A,North York,Lawrence Heights
M6A,North York,Lawrence Manor
M7A,Queen's Park,Queens Park
M9A,Etobicoke,Islington Avenue
M1B,Scarborough,Rouge


In [24]:
#Group rows with similar Postalcode
table_df = table_df.groupby(["Postalcode", "Borough"]).agg(lambda col: ", " .join(col))

In [25]:
table_df.head()

,,Neighbourhood
Postalcode,Borough,
M1B,Scarborough,"Rouge, Malvern"
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


In [26]:
table_df.reset_index(["Postalcode", "Borough"], inplace = True)

In [27]:
table_df.head(5)

,Postalcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [317]:
table_df.shape

(103, 3)

In [30]:
#Read csv file for co-ordinates
latlon_df = pd.read_csv("https://cocl.us/Geospatial_data")

In [33]:
latlon_df.rename(columns = {"Postal Code":"Postalcode"}, inplace = True)

In [36]:
#Merge both tables to obtain table with co-ordinates
new_table_df = pd.merge(table_df, latlon_df, on = "Postalcode")

In [39]:
new_table_df.head(20)

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [44]:
import folium

In [48]:
#Create table 
toronto_table = new_table_df[new_table_df["Borough"].str.contains("Toronto")]

In [62]:
latitude = 43.7280
longitude = -79.3888

In [54]:
toronto_table

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
45,M4P,Central Toronto,Davisville North,43.712751,-79.390197
46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
47,M4S,Central Toronto,Davisville,43.704324,-79.388790
48,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
49,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049


In [66]:
map_toronto = folium.Map(location = [latitude, longitude], zoom_start = 11)

for lat, lng, label in zip(toronto_table["Latitude"], toronto_table["Longitude"], toronto_table["Neighbourhood"]):
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
    [lat, lng],
    radius = 5,
    popup = label,
    color = "blue",
    fill = True,
    fill_color = "#3186cc",
    fill_opacity = 0.7,
    parse_html = False).add_to(map_toronto)

map_toronto
    